In [13]:
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
#from sklearn.model_selection import train_test_split
import tensorflow as tf
import librosa
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#import seaborn as sns
import numpy as np
from pydub import AudioSegment
from pydub.playback import play
from scipy.io import wavfile
from tempfile import mktemp

from datasets import load_dataset

In [ ]:
#TODO
#1. Do the data normalizing thing. you know you gotta
#2. I think you need a better loss function- something that accounts for getting closer
#3. Test different timesteps
#4. Test different epochs
#5. Test differet batch sizes

#Original: Timestep = 2 Epoch = 2(?) Batch = 2(?)

In [14]:
TIME_STEPS = 5
LETTER_CONVERSION = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q','r','s','t','u','v','w','x','y','z',' ','.', '' ]
image_name = 0
min_transcript = 100

x_processed_data = []
y_processed_data = []

In [15]:
ds = load_dataset("mozilla-foundation/common_voice_11_0", "en", split='train[:100]')
print(ds[33])

Found cached dataset common_voice_11_0 (/home/hetricke/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/en/11.0.0/3f27acf10f303eac5b6fbbbe02495aeddb46ecffdb0a2fe3507fcfbf89094631)


{'client_id': '1b488ea5d34be1930a2ddebc8246bb145ee4427735f36cd53cbe7bdddba1a4d3fee5034657c42fa813560d5135db6d37efb90ccbf63d24003c4692d95ecc09f7', 'path': '/home/hetricke/.cache/huggingface/datasets/downloads/extracted/6a3d5b8fc2d9769c190e0837d10a49ba8ec8047259e8c2f55964fc9ccb4b3969/en_train_0/common_voice_en_18588151.mp3', 'audio': {'path': '/home/hetricke/.cache/huggingface/datasets/downloads/extracted/6a3d5b8fc2d9769c190e0837d10a49ba8ec8047259e8c2f55964fc9ccb4b3969/en_train_0/common_voice_en_18588151.mp3', 'array': array([ 0.00000000e+00,  1.74605392e-15,  3.24473335e-15, ...,
       -7.94060179e-06, -1.55025424e-04, -1.14534341e-04]), 'sampling_rate': 48000}, 'sentence': 'I mentor junior developers and train them to be confident.', 'up_votes': 2, 'down_votes': 1, 'age': '', 'gender': '', 'accent': '', 'locale': 'en', 'segment': ''}


In [16]:
def detect_leading_silence(sound, silence_threshold=-50.0, chunk_size=10):
    trim_ms = 0 # ms

    assert chunk_size > 0 # to avoid infinite loop
    while sound[trim_ms:trim_ms+chunk_size].dBFS < silence_threshold and trim_ms < len(sound):
        trim_ms += chunk_size

    return trim_ms

In [17]:
def convertToNum(s):
    try: 
        return LETTER_CONVERSION.index(s)
    except:
        return -1

In [18]:
def convertToLetter(val):
    return LETTER_CONVERSION[val]

In [19]:
def transcript_prep(transcription):

    global min_transcript     

    min_transcript = min_transcript if (min_transcript < len(transcription)) else len(transcription)

    if transcription.startswith('"') and transcription.endswith('"'):
        # we can remove trailing quotation marks as they do not affect the transcription
        transcription = transcription[1:-1]

    transcription = transcription.lower()
    
    if len(transcription) > 0:
        if transcription[-1] not in [".", "?", "!"]:
            # append a full-stop to sentences that do not end in punctuation
            transcription = transcription + "."

    transcript_pieces = [ord(c) for c in transcription]

    return transcript_pieces


In [20]:
def prepare_dataset(batch):

    global image_name
    global x_processed_data
    global y_processed_data

    #reads in mp3
    mp3_audio = AudioSegment.from_file(batch['path'], format="mp3")  # read mp3

    if(round(mp3_audio.duration_seconds) == 0):
        return batch

    #removes silent audio from the beginning and end
    start_trim = detect_leading_silence(mp3_audio)
    end_trim = detect_leading_silence(mp3_audio.reverse())
    duration = len(mp3_audio)    
    trimmed_sound = mp3_audio[start_trim:duration-end_trim]

    #converts the mp3 into a wav file
    wname = mktemp('.wav')  # use temporary file
    trimmed_sound.export(wname, format="wav")  # convert to wav
    FS, audio_data = wavfile.read(wname)  # read wav file


    #creates a file name for the spectrogram
    file_name = "images/"+ str(image_name) + ".png"
    image_name = image_name + 1

    #creates and saves the spectrogram
    plt.figure()
    plt.specgram(audio_data, Fs=FS, NFFT=128, noverlap=0)  # plot
    plt.axis('off')
    plt.savefig(file_name, bbox_inches='tight')

    #clears the figure for the next audio transcript- otherwise it just overwrites the image, which causes Problems
    plt.close()


    #loads the spectrogram and turns it into an array
    img = keras.preprocessing.image.load_img(file_name)
    img_array = keras.preprocessing.image.img_to_array(img)


    #slices the image array into appriopriate sizes for each chunk
    samples = len(batch["sentence"])
    features = int(img_array.shape[0]/samples)

    
    divided_img_array = []
    x_data = []
    index = 0

    for i in range(samples):
        feature_array = img_array[index:index+features]
        new_dim = feature_array.shape[0]*feature_array.shape[1]
        feature_array = feature_array.reshape(new_dim, -1)

        original_length = feature_array.shape[0]
        padding = 11000-original_length
        feature_array = np.pad(feature_array, [(0,padding), (0,0)], mode='constant')
        feature_array = feature_array.flatten()

        feature_array = feature_array.tolist()
        
        divided_img_array.insert(0,feature_array)


        if(len(divided_img_array)< TIME_STEPS):

            info = divided_img_array.copy()

            null_list = [0]*(len(feature_array))

            while len(info) < TIME_STEPS:
                info.append(null_list.copy())
    
            x_data.append(info.copy())

        else:
            x_data.append(divided_img_array[:TIME_STEPS].copy())

        
        index = index + features

    x_processed_data.append(x_data)
    y_processed_data.append(transcript_prep(batch["sentence"]))
    


    #reshape the spectogram into a 3d array that goes (num samples x timesteps x features)
    #1 sample is how many times this gets run through
    #timesteps is how far backward to grab (so that's a value that can be played with!)
    #num samples is I think how long it is (so grab the mp3 length)
    #slice the image accordingly
    #convert the image into an array (reference the obj detection) to use for the features
    #profit

    #so the min time is three, which, at 515 timesteps, gets 171.6666 columns of data per second
    

    return batch


In [21]:
image_name = 0
x_processed_data = []
y_processed_data = []
ds = ds.map(prepare_dataset, desc="preprocess dataset")


preprocess dataset:  29%|██▉       | 29/100 [00:10<00:22,  3.15 examples/s]/home/hetricke/anaconda3/envs/speech_recog_env/lib/python3.11/site-packages/matplotlib/axes/_axes.py:7939: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)
preprocess dataset:  64%|██████▍   | 64/100 [00:19<00:08,  4.17 examples/s]/home/hetricke/anaconda3/envs/speech_recog_env/lib/python3.11/site-packages/matplotlib/axes/_axes.py:7939: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)
preprocess dataset:  83%|████████▎ | 83/100 [00:24<00:04,  3.74 examples/s]/home/hetricke/anaconda3/envs/speech_recog_env/lib/python3.11/site-packages/matplotlib/axes/_axes.py:7939: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)
preprocess dataset:  85%|████████▌ | 85/100 [00:24<00:03,  3.99 examples/s]/home/hetricke/anaconda3/envs/speech_recog_env/lib/python3.11/site-packages/matplotlib/axes/_axes.py:7939: RuntimeWarning: divide by zero enco

In [22]:

x_train = np.array(x_processed_data[:98])
y_train = np.array(y_processed_data[:98])

x_test = np.array(x_processed_data[99], dtype='int')
y_test = np.array(y_processed_data[99], dtype='int')


/tmp/ipykernel_10383/1399697840.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train = np.array(x_processed_data[:98])
/tmp/ipykernel_10383/1399697840.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_train = np.array(y_processed_data[:98])


In [23]:
#build model
model = keras.Sequential()
model.add(keras.layers.SimpleRNN(units = 128, activation = 'relu',input_shape = (TIME_STEPS, 33000)))
model.add(keras.layers.Dense(units = 128, activation = 'relu'))
model.add(keras.layers.Dense(units = 128, activation='relu'))
model.add(keras.layers.Dense(units = 128, activation ='relu'))
model.add(keras.layers.Dense(len(LETTER_CONVERSION), activation = 'softmax'))
model.compile(optimizer='adam', metrics=['accuracy'], loss='mean_squared_error')

In [24]:
#
for recording in range(len(x_train)):
    model.fit(x_train[recording], y_train[recording], epochs=10, batch_size=5)


Epoch 1/10
12/12 [==============================] - 2s 75ms/step - loss: 9780.3340 - accuracy: 0.0000e+00
Epoch 2/10
12/12 [==============================] - 1s 77ms/step - loss: 9780.3340 - accuracy: 0.0000e+00
Epoch 3/10
12/12 [==============================] - 1s 81ms/step - loss: 9780.3340 - accuracy: 0.0000e+00
Epoch 4/10
12/12 [==============================] - 1s 78ms/step - loss: 9780.3340 - accuracy: 0.0000e+00
Epoch 5/10
12/12 [==============================] - 1s 76ms/step - loss: 9780.3330 - accuracy: 0.0000e+00
Epoch 6/10
12/12 [==============================] - 1s 81ms/step - loss: 9780.3340 - accuracy: 0.0000e+00
Epoch 7/10
12/12 [==============================] - 1s 75ms/step - loss: 9780.3350 - accuracy: 0.0000e+00
Epoch 8/10
12/12 [==============================] - 1s 85ms/step - loss: 9780.3340 - accuracy: 0.0000e+00
Epoch 9/10
12/12 [==============================] - 1s 81ms/step - loss: 9780.3340 - accuracy: 0.0000e+00
Epoch 10/10
12/12 [===========================

In [ ]:
#add computer i/o for speech recognition